In [ ]:
import datasets

#load the japanese dataset
dataset_ja = datasets.load_dataset("cc100", lang="ja")

In [ ]:
# sample an element
dataset_ja["train"][961563]

In [ ]:
from src.make_datasets import make_sentence_files

make_sentence_files(dataset_ja["train"])

In [ ]:
# read text files as a list

import glob
import cfg

jp_sentence_files = glob.glob(cfg.JP_SENTENCES_DIR + "/*.txt")
print(jp_sentence_files)
jp_sentence_files=jp_sentence_files[0]

In [ ]:
!pip install sentencepiece

In [ ]:
#load the english dataset
#dataset_ja = datasets.load_dataset("cc100", lang="en")

In [ ]:
jp_sentence_files

In [ ]:
import sentencepiece as spm

#spm.SentencePieceTrainer.train(input=jp_sentence_files, 
#                               model_prefix='m', vocab_size=1000)

spm.SentencePieceTrainer.train(input=jp_sentence_files
                                    , model_prefix="m"
                                    , vocab_size=4000
                                    , character_coverage = 0.995
                                    , num_threads=60
                                    , train_extremely_large_corpus=True
                                      ) 

In [ ]:
# changing the ja into jp here :)
import cfg
import sentencepiece as spm
import time
#from multiprocessing import Pool


languages = ["jp"]#, "en"]
sentences_fp = {"jp": jp_sentence_files}#, "en": cfg.EN_SENTENCES_FILE}
vocab_sizes = [4000, 8000, 16000, 24000, 32000, 48000]
character_coverages = {"jp": 0.995, "en": 1.00}
#multiproc_pool_size = 6
num_threads = 60

def train_single_param(lang, vocab_size):
    """ Train a single parameter combination"""
    
    start = time.time()
    model_prefix = "cc100_"+lang + "_vocab_" + str(vocab_size)
    spm.SentencePieceTrainer.train(input=sentences_fp[lang]
                                       , model_prefix=model_prefix
                                       , vocab_size=vocab_size
                                       , character_coverage = character_coverages[lang]
                                       , num_threads=num_threads
                                       , minloglevel=1
                                      ) 
    print("Trained {} in {} seconds".format(model_prefix, time.time()-start))
    


#loop over parameters for the actual train in multiprocessing pool
param_combinations = [(lang,vocab_size) for lang in languages for vocab_size in vocab_sizes]
#p = Pool(multiproc_pool_size)

#p.starmap(train_single_param, param_combinations)

#for lang, vocab_size in param_combinations:
#    train_single_param(lang, vocab_size)
train_single_param(param_combinations[0][0],param_combinations[0][1])